In [5]:
import pickle
import glob
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import log_loss
from xgboost import XGBClassifier

# load feats
train_x,test_x = [],[]
for feat in glob.glob('../features/*.pkl'):
#     if 'tfidf' in feat or 'lr' in feat or 'mnb' in feat:
#         continue
    print('file path',feat)
    a,b = pickle.load(open(feat,'rb'))
    print(a.shape,b.shape)
    train_x.append(a)
    test_x.append(b)
train_x = np.nan_to_num(np.hstack(train_x))
test_x = np.nan_to_num(np.hstack(test_x))
print(train_x.shape)
    
# load y
train = pd.read_csv("../input/train.csv")
list_classes = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
train_y = train[list_classes].values.astype('int')
print(train_x.shape)

file path ../features/lr_feat1.pkl
(95851, 6) (226998, 6)
file path ../features/cnn_1_feat.pkl
(95851, 6) (226998, 6)
file path ../features/mnb_feat1.pkl
(95851, 6) (226998, 6)
file path ../features/other_feat.pkl
(95851, 16) (226998, 16)
file path ../features/mnb_feat2.pkl
(95851, 6) (226998, 6)
file path ../features/nn_clean_1_feat.pkl
(95851, 6) (226998, 6)
file path ../features/tfidf_feat1.pkl
(95851, 30) (226998, 30)
file path ../features/cnn_2_feat.pkl
(95851, 6) (226998, 6)
file path ../features/nn_2_feat.pkl
(95851, 6) (226998, 6)
file path ../features/lstm_1_feat.pkl
(95851, 6) (226998, 6)
file path ../features/lr_feat2.pkl
(95851, 6) (226998, 6)
file path ../features/cnn_clean_1_feat.pkl
(95851, 6) (226998, 6)
file path ../features/nn_1_feat.pkl
(95851, 6) (226998, 6)
file path ../features/tfidf_feat2.pkl
(95851, 30) (226998, 30)
(95851, 142)
(95851, 142)


In [6]:
print(train_y[:10])
print(len(train_y))
print(sum(train_y[:,0]))

[[1 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]]
95851
9237


In [7]:
from sklearn.model_selection import KFold
def simple_ens(model_name,k=3):
    kf = KFold(n_splits=k, shuffle=True, random_state=42)
    test_pred = np.zeros((226998,6))
    all_train_loss_l,all_val_loss_l = 0,0
    for train_index, test_index in kf.split(train_x):
        # x,y
        curr_x,curr_y = train_x[train_index],train_y[train_index]
        hold_out_x,hold_out_y = train_x[test_index],train_y[test_index]
        val_loss_l,train_loss_l = 0,0
        
        # train for each class
        for i in range(6):
            if model_name == 'lr':
                model = LogisticRegression()
            if model_name == 'xgb':
                model = XGBClassifier()
            model.fit(curr_x,curr_y[:,i])
            print(i)
            try:
                curr_train_loss = log_loss(curr_y[:,i],model.predict_proba(curr_x))
                curr_val_loss = log_loss(hold_out_y[:,i],model.predict_proba(hold_out_x))
                print(curr_train_loss,curr_val_loss)
                val_loss_l += curr_val_loss
                train_loss_l += curr_train_loss
            except:
                pass
            curr_test_pred = model.predict_proba(test_x)[:,1]
            test_pred[:,i] += curr_test_pred
            
        # avg 6 class
        train_loss_l = train_loss_l/6
        val_loss_l = val_loss_l/6
        print('this fold avg train',train_loss_l,'avg val',val_loss_l)
        
        # avg k fold
        all_train_loss_l += train_loss_l/k
        all_val_loss_l += val_loss_l/k
        print('========================')
    test_pred = test_pred/k
    print('all train avg',all_train_loss_l,'all val avg',all_val_loss_l)
    return test_pred

print('done')
            
                
            

done


In [9]:
xgb_res = simple_ens('xgb')
sample_submission = pd.read_csv("../input/sample_submission.csv")
sample_submission[list_classes] = xgb_res
sample_submission.to_csv("../results/xgb_ens_csv.gz", index=False, compression='gzip')
print(sample_submission.head())
print('save done')

0
0.0904575774248 0.0975692401024
1
0.0186695996197 0.0228616965284
2
0.0467911080421 0.0502222376972
3
0.00935013589221 0.013662158496
4
0.0597594016982 0.0690884261564
5
0.0242196736192 0.0264549772495
this fold avg train 0.0415412493827 avg val 0.046643122705
0
0.0893438847042 0.0992942597412
1
0.0189930590893 0.0227402921149
2
0.0457216146413 0.0527207624096
3
0.00937742294745 0.0133596948236
4
0.0614368068798 0.0655567427119
5
0.0226814802397 0.0294618937849
this fold avg train 0.0412590447503 avg val 0.047188940931
0
0.0909900858071 0.0955352995305
1
0.0184835911431 0.0236736499654
2
0.0452108140987 0.0529192701323
3
0.00991140130186 0.0122183754017
4
0.0612944571837 0.0655469527864
5
0.0236269922671 0.0285991054779
this fold avg train 0.0415862236336 avg val 0.0464154422157
all train avg 0.0414621725889 all val avg 0.0467491686172
         id     toxic  severe_toxic   obscene    threat    insult  \
0   6044863  0.005982      0.000065  0.001045  0.000195  0.001259   
1   6102620 